In [5]:
dataset = "cifar100"
iterations = "2,2,2,2"
u_channels = "256,256,256,256"
f_channels = "256,256,256,256"
batch_size = 128
epochs = 150
epoch_step = 30
lr = .1
lr_step = 10
momentum = .9
wd = .0005
graph = True

iterations = [int(x) for x in iterations.split(",")]
u_channels = [int(x) for x in u_channels.split(",")]
f_channels = [int(x) for x in f_channels.split(",")]

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import numpy as np
import logging
tf.get_logger().setLevel(logging.ERROR)
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    dataset,
    split = ["train", "test"],
    as_supervised = True,
    with_info = True)

rescale = tf.keras.layers.Rescaling(1./255)
if dataset == "mnist":
  mean, variance = [.1307], np.square([.3081])
if dataset == "cifar10":
  mean, variance = [.4914, .4822, .4465], np.square([.2023, .1994, .2010])
if dataset == "cifar100":
  mean, variance = [.5071, .4865, .4409], np.square([.2673, .2564, .2762])
normalize = tf.keras.layers.Normalization(mean = mean,
                                          variance = variance)

def preprocess(ds, training):
  if training:
    layers = tf.keras.Sequential([
      rescale,
      tf.keras.layers.RandomTranslation(height_factor = .125,
                                        width_factor = .125,
                                        fill_mode = "constant"),
      tf.keras.layers.RandomFlip(mode = "horizontal"),
      normalize
    ])
    ds = ds.shuffle(ds_info.splits["train"].num_examples)
  else:
    layers = tf.keras.Sequential([rescale, normalize])

  ds = ds.batch(batch_size)
  ds = ds.map(lambda x, y: (layers(x), y),
              num_parallel_calls = tf.data.AUTOTUNE)
  ds = ds.cache()
  ds = ds.prefetch(tf.data.AUTOTUNE)

  return ds

ds_train = preprocess(ds_train, training = True)
ds_test = preprocess(ds_test, training = False)

In [ ]:
class MgSmooth(tf.keras.layers.Layer):

  def __init__(self, iterations, u_channels, f_channels):
    super(MgSmooth, self).__init__()

    self.iterations = iterations
    self.A = tf.keras.layers.Conv2D(u_channels,
                                    (3, 3),
                                    strides = (1, 1),
                                    padding = "same",
                                    use_bias = False,
                                    kernel_initializer = "he_uniform")
    self.B = tf.keras.layers.Conv2D(f_channels,
                                    (3, 3),
                                    strides = (1, 1),
                                    padding = "same",
                                    use_bias = False,
                                    kernel_initializer = "he_uniform")

    self.A_bns, self.B_bns = [], []
    for _ in range(self.iterations):
      self.A_bns.append(tf.keras.layers.BatchNormalization())
      self.B_bns.append(tf.keras.layers.BatchNormalization())

  def call(self, u, f):
    for i in range(self.iterations):
      error = tf.nn.relu(self.A_bns[i](f - self.A(u)))
      u = u + tf.nn.relu(self.B_bns[i](self.B(error)))
    return u, f

class MgBlock(tf.keras.layers.Layer):

  def __init__(self, iterations, u_channels, f_channels, A_old):
    super(MgBlock, self).__init__()

    self.iterations = iterations
    self.Pi = tf.keras.layers.Conv2D(u_channels,
                                     (3, 3),
                                     strides = (2, 2),
                                     padding = "same",
                                     use_bias = False,
                                     kernel_initializer = "he_uniform")
    self.R = tf.keras.layers.Conv2D(f_channels,
                                    (3, 3),
                                    strides = (2, 2),
                                    padding = "same",
                                    use_bias = False,
                                    kernel_initializer = "he_uniform")
    self.A_old = A_old
    self.MgSmooth = MgSmooth(self.iterations, u_channels, f_channels)

    self.Pi_bn = tf.keras.layers.BatchNormalization()
    self.R_bn = tf.keras.layers.BatchNormalization()

  def call(self, u0, f0):
    u1 = tf.nn.relu(self.Pi_bn(self.Pi(u0)))
    error = tf.nn.relu(self.R_bn(self.R(f0 - self.A_old(u0))))
    f1 = error + self.MgSmooth.A(u1)
    u, f = self.MgSmooth(u1, f1)
    return u, f

class MgNet(tf.keras.Model):

  def __init__(self, iterations, u_channels, f_channels, in_shape, out_shape):
    super(MgNet, self).__init__()

    self.iterations = iterations
    self.in_shape = in_shape
    self.A_init = tf.keras.layers.Conv2D(u_channels[0],
                                         (3, 3),
                                         strides = (1, 1),
                                         padding = "same",
                                         use_bias = False,
                                         kernel_initializer = "he_uniform")
    self.A_bn = tf.keras.layers.BatchNormalization()

    self.blocks = []
    for i in range(len(self.iterations)):
      if i == 0:
        self.blocks.append(MgSmooth(iterations[i],
                                    u_channels[i],
                                    f_channels[i]))
        continue
      if i == 1:
        self.blocks.append(MgBlock(iterations[i],
                                   u_channels[i],
                                   f_channels[i],
                                   self.blocks[0].A))
        continue
      self.blocks.append(MgBlock(iterations[i],
                                 u_channels[i],
                                 f_channels[i],
                                 self.blocks[i - 1].MgSmooth.A))

    x = in_shape[0]
    for i in range(len(self.blocks) - 1):
      x = ((x + 2 - 3) // 2) + 1
    self.pool = tf.keras.layers.AveragePooling2D(pool_size = (x, x))
    self.fc = tf.keras.layers.Dense(out_shape,
                                    activation = "softmax")
  
  def call(self, u0):
    f = tf.nn.relu(self.A_bn(self.A_init(u0)))
    u = tf.multiply(f, 0)

    for block in self.blocks:
      u, f = block(u, f)
    u = self.pool(u)
    u = tf.squeeze(u, [-2, -3])
    u = self.fc(u)
    return u

In [ ]:
def lr_schedule(epoch, lr):
  if (epoch + 1) % epoch_step == 0:
    return lr / lr_step
  return lr

tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices("GPU")
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
  model = MgNet(iterations = iterations,
                u_channels = u_channels,
                f_channels = f_channels,
                in_shape = ds_info.features["image"].shape,
                out_shape = ds_info.features["label"].num_classes)

  loss = tf.keras.losses.SparseCategoricalCrossentropy()

  lr_s = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
  optimizer = tfa.optimizers.SGDW(learning_rate = lr,
                                  weight_decay = wd,
                                  momentum = momentum)
  
  model.compile(optimizer = optimizer,
                loss = loss,
                metrics = ["accuracy"])

  history = model.fit(ds_train,
                      epochs = epochs,
                      validation_data = ds_test,
                      callbacks = [lr_s])

model.summary()

In [ ]:
if graph:
  loss = history.history["loss"]
  accuracy = history.history["accuracy"]
  val_loss = history.history["val_loss"]
  val_accuracy = history.history["val_accuracy"]
  timerange = range(len(loss))

  fig,ax = plt.subplots()
  train_loss_plot, = ax.plot(timerange, loss, color = "blue")
  val_loss_plot, = ax.plot(timerange, val_loss, color = "cyan")
  train_loss_plot.set_label("Train Loss")
  val_loss_plot.set_label("Validation Loss")
  ax.set_xlabel("Epoch")
  ax.set_ylabel("Loss")
  ax.legend(loc = "upper left")
  ax2 = ax.twinx()
  train_acc_plot, = ax2.plot(timerange, accuracy, color = "purple")
  val_acc_plot, = ax2.plot(timerange, val_accuracy, color = "pink")
  train_acc_plot.set_label("Train Accuracy")
  val_acc_plot.set_label("Validation Accuracy")
  ax2.set_ylabel("Accuracy")
  ax2.legend(loc = "upper right")
  plt.title("Loss vs Accuracy")
  plt.savefig(f"{dataset}_mgnet_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.png")

# **PyTorch Test**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from timeit import default_timer as timer
import numpy as np
import logging
import os

In [2]:
class MgIte(nn.Module):
  def __init__(self, A, B):
    super().__init__()
    self.A = A
    self.B = B        
    self.bn1 =nn.BatchNorm2d(A.weight.size(0))
    self.bn2 =nn.BatchNorm2d(B.weight.size(0))

  def forward(self, out):
    u, f = out
    u = u + F.relu(self.bn2(self.B(F.relu(self.bn1((f-self.A(u)))))))
    out = (u, f)
    return out
    
class MgRestriction(nn.Module):
  def __init__(self, A_old, A_conv, Pi_conv, R_conv):
    super().__init__()
    self.A_old = A_old
    self.A_conv = A_conv
    self.Pi_conv = Pi_conv
    self.R_conv = R_conv

    self.bn1 = nn.BatchNorm2d(Pi_conv.weight.size(0))
    self.bn2 = nn.BatchNorm2d(A_old.weight.size(0))

  def forward(self, out):
    u_old, f_old = out
    u = F.relu(self.bn1(self.Pi_conv(u_old)))
    f = F.relu(self.bn2(self.R_conv(f_old-self.A_old(u_old)))) + self.A_conv(u)
    out = (u,f)
    return out

class MgNet(nn.Module):
  def __init__(self, args,num_classes):
    super().__init__()
    self.num_iteration = args.num_ite
    self.args = args
    
    # inilization layer
    if args.dataset == 'mnist':
      self.num_channel_input=1
    else:
      self.num_channel_input=3
    self.conv1 = nn.Conv2d(self.num_channel_input, args.num_channel_f, kernel_size=3, stride=1,\
                            padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(args.num_channel_f)        

    A_conv = nn.Conv2d(args.num_channel_u, args.num_channel_f, kernel_size=3, stride=1, padding=1, bias=False)
    if not args.wise_B:
      B_conv = nn.Conv2d(args.num_channel_f, args.num_channel_u, kernel_size=3,stride=1, padding=1, bias=False)

    layers = []
    for l, num_iteration_l in enumerate(self.num_iteration):
      for i in range(num_iteration_l):
        if args.wise_B:
          B_conv = nn.Conv2d(args.num_channel_f, args.num_channel_u, kernel_size=3,\
                              stride=1, padding=1, bias=False)
        
        layers.append(MgIte(A_conv, B_conv))

      setattr(self, 'layer'+str(l), nn.Sequential(*layers))


      if l < len(self.num_iteration)-1:
        A_old = A_conv
        #B_old = B_conv
        A_conv = nn.Conv2d(args.num_channel_u, args.num_channel_f, kernel_size=3,\
                            stride=1, padding=1, bias=False)
        if not args.wise_B:
          B_conv = nn.Conv2d(args.num_channel_f, args.num_channel_u, kernel_size=3,\
                              stride=1, padding=1, bias=False)
        Pi_conv = nn.Conv2d(args.num_channel_u, args.num_channel_u, kernel_size=3,\
                            stride=2, padding=1, bias=False)
        R_conv = nn.Conv2d(args.num_channel_f, args.num_channel_u, kernel_size=3, \
                            stride=2, padding=1, bias=False)
        layers= [MgRestriction(A_old, A_conv, Pi_conv, R_conv)]

    self.pooling = nn.AdaptiveAvgPool2d(1)
    self.fc = nn.Linear(args.num_channel_u ,num_classes)

  def forward(self, u):
    f = F.relu(self.bn1(self.conv1(u)))
    if torch.cuda.is_available():
      u = torch.zeros(f.size(),device=torch.device('cuda'))
    else:
      u = torch.zeros(f.size())
    out = (u, f)

    for l in range(len(self.num_iteration)):
      out = getattr(self, 'layer'+str(l))(out)
    u,f = out
    u = self.pooling(u)
    u = u.view(u.shape[0], -1)
    u = self.fc(u)
    return u

In [3]:
def adjust_learning_rate(optimizer, epoch, init_lr):
  #lr = 1.0 / (epoch + 1)
  lr = init_lr * 0.1 ** (epoch // 50)
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr
  return lr

def load_data(path,minibatch_size,dataset):
  if dataset=='mnist':
    trainloader = torch.utils.data.DataLoader(torchvision.datasets.MNIST(root=path, train=True, download=True,
                                      transform=torchvision.transforms.Compose([
                                      torchvision.transforms.ToTensor(),
                                      torchvision.transforms.Normalize(
                                      (0.1307,), (0.3081,))
                                      ])),
                                      batch_size=minibatch_size, num_workers=4,shuffle=True)
                                      
    testloader = torch.utils.data.DataLoader(torchvision.datasets.MNIST(root=path, train=False, download=True,
                                      transform=torchvision.transforms.Compose([
                                      torchvision.transforms.ToTensor(),
                                      torchvision.transforms.Normalize(
                                      (0.1307,), (0.3081,))
                                      ])),
                                      batch_size=minibatch_size, num_workers=4,shuffle=True)
    num_classes = 10
  if dataset=='cifar10':
    normalize = torchvision.transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
    transform_train = torchvision.transforms.Compose([torchvision.transforms.RandomCrop(32, padding=4),\
                                    torchvision.transforms.RandomHorizontalFlip(),\
                                    torchvision.transforms.ToTensor(),\
                                    normalize])

    transform_test  = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),normalize])


    trainset = torchvision.datasets.CIFAR10(root=path, train=True,download=True,transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=minibatch_size,num_workers=4, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root=path, train=False, download=True,transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=minibatch_size,num_workers=4, shuffle=False)
    num_classes = 10
  if dataset=='cifar100':
    normalize = torchvision.transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762))
    transform_train = torchvision.transforms.Compose([torchvision.transforms.RandomCrop(32, padding=4),\
                                    torchvision.transforms.RandomHorizontalFlip(),\
                                    torchvision.transforms.ToTensor(),\
                                    normalize])

    transform_test  = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),normalize])


    trainset = torchvision.datasets.CIFAR100(root=path, train=True,download=True,transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=minibatch_size,num_workers=4, shuffle=True)

    testset = torchvision.datasets.CIFAR100(root=path, train=False, download=True,transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=minibatch_size,num_workers=4, shuffle=False)
    num_classes = 100
  
  return trainloader,testloader,num_classes

def train_process(model,save_name,num_epochs,lr,trainloader,testloader):
  test_acc = []
  if use_cuda:
    model = model.cuda()
  criterion = nn.CrossEntropyLoss()
  # optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay = 0.0005)
  optimizer = optim.SGD(model.parameters(), lr=lr)
  print("total {} paramerters".format(sum(x.numel() for x in model.parameters())))

  start = timer()
  Test_acc = 0
  for epoch in range(1,num_epochs+1):
    current_lr = adjust_learning_rate(optimizer, epoch, lr)

    model.train()
    for i, (images, labels) in enumerate(trainloader):
      if use_cuda:
        images = images.cuda()
        labels = labels.cuda()

      # Forward pass to get the loss
      outputs = model(images) 
      loss = criterion(outputs, labels)

      # Backward and compute the gradient
      optimizer.zero_grad()
      loss.backward()  #backpropragation
      optimizer.step() #update the weights/parameters

    # Training accuracy 
    model.eval()
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(trainloader):
      with torch.no_grad():
        if use_cuda:
          images = images.cuda()
          labels = labels.cuda()
        outputs = model(images)
        p_max, predicted = torch.max(outputs, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum()
    training_accuracy = float(correct)/total

    # Test accuracy
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(testloader):
      with torch.no_grad():
        if use_cuda:
          images = images.cuda()
          labels = labels.cuda()
        outputs = model(images)
        p_max, predicted = torch.max(outputs, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum()
    test_accuracy = float(correct)/total
    test_acc.append(test_accuracy)
    if test_accuracy>Test_acc:
      # save(model, optimizer,save_name,epoch)
      Test_acc = test_accuracy

    print('Epoch:{}, loss:{:.2f}, lr:{:.6f}, train acc:{:.4f}, test acc:{:.4f}, best acc:{:.4f}'.\
           format(epoch,loss,current_lr,training_accuracy,test_accuracy,np.max(test_acc)))

  #end = timer()
  print('last_train_acc:{:.4f},max_acc:{:.4f},last_acc:{:.4f}'.format(training_accuracy,np.max(test_acc),test_accuracy))
  return training_accuracy,np.max(test_acc),test_accuracy

In [ ]:
class args_obj():
  def __init__(self):
    self.dataset = "cifar100"
    self.path = "~/pytorch_datasets"
    self.num_ite = "2,2,2,2"
    self.num_channel_u = 256
    self.num_channel_f = 256
    self.wise_B = False
    self.minibatch_size = 128
    self.num_epoch = 300
    self.lr = .1
    
args = args_obj()
args.num_ite = [int(i) for i in args.num_ite.split(',')]

use_cuda = torch.cuda.is_available()
trainloader,testloader,num_classes = load_data(args.path,args.minibatch_size,args.dataset)
model = MgNet(args,num_classes=num_classes)
if use_cuda:
  model =model.cuda()
train_acc,test_max_acc,test_last_acc = train_process(model,"",args.num_epoch,args.lr,trainloader,testloader)